## Calculate Phi vertical mean for a particular layer and LME

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.pyplot as plt

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'

In [3]:
filename = f'{path}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_monthly.nc'
dset = xr.open_dataset(filename)
dz = dset['dz']/1e2 # m
darea = dset['TAREA']/1e10 #km*km
dvol = dz*darea

In [4]:
KMT = dset['KMT']
nlat = dset.dims['nlat']
nlon = dset.dims['nlon']
for xx in range(nlat):
    for yy in range(nlon):
        kk = np.int(KMT[xx, yy])
        if kk < 60:
            dvol[kk:, xx, yy] = float("nan")

In [5]:
#filename = f'{path}/DPLE-FOSI_hindcast_Phi_1948-2017_monthly.nc'
filename = f'{path}/DPLE-FOSI_hindcast_Phi_Eo_space_1948-2017_monthly.nc'
ds = xr.open_dataset(filename)
dsf = xr.Dataset()
for ii in range(13):
    print(ii)
    Phi_i = ds[f'Phi_{ii}']*dvol
    # select a vertical layer
    dsf[f'Phi_{ii}_0-200m'] = Phi_i.isel(z_t=range(0, 20, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(0, 20, 1)).sum(dim='z_t'))
    dsf[f'Phi_{ii}_200-600m'] = Phi_i.isel(z_t=range(20, 35, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(20, 35, 1)).sum(dim='z_t'))

0
1
2
3
4
5
6
7
8
9
10
11
12


In [6]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:          (nlat: 384, nlon: 320, time: 840)
Coordinates:
    TLAT             (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.19 72.19
    TLONG            (nlat, nlon) float64 320.6 321.7 322.8 ... 319.4 319.8
    ULAT             (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.41 72.41
    ULONG            (nlat, nlon) float64 321.1 322.3 323.4 ... 319.6 320.0
  * time             (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:00:00
Dimensions without coordinates: nlat, nlon
Data variables: (12/26)
    Phi_0_0-200m     (time, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_0_200-600m   (time, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_1_0-200m     (time, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_1_200-600m   (time, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_2_0-200m     (time, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_2_200-600m   (time, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    ...               ...
    Phi_10_0-200m    (time, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_10_200-600m  (time, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_11_0-200m    (time, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_11_200-600m  (time, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_12_0-200m    (time, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_12_200-600m  (time, nlat, nlon) float64 nan nan nan nan ... nan nan nan

In [7]:
%time
#dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_vertical_mean_1948-2017_monthly.nc', mode='w')
dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_Eo_space_vertical_mean_1948-2017_monthly.nc', mode='w')

CPU times: user 2 µs, sys: 4 µs, total: 6 µs
Wall time: 233 µs


## calculate LME-averaged Phi 

In [3]:
path = f'/glade/scratch/{USER}/DPLE-results'
#filename = f'{path}/DPLE-FOSI_hindcast_Phi_vertical_mean_1948-2017_monthly.nc'
filename = f'{path}/DPLE-FOSI_hindcast_Phi_Eo_space_vertical_mean_1948-2017_monthly.nc'
dsf = xr.open_dataset(filename)

In [4]:
datadir = f'/glade/u/home/{USER}/O2prediction/Prediction/notebooks/data'
filename = '%s/LME-mask-POP_gx1v6.nc' % datadir
ds1 = xr.open_dataset(filename)
lme_mask = ds1['lme_mask']

In [6]:
#lme = range(np.int(lme_mask.min()), np.int(lme_mask.max()), 1)
lme = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 65]
nmon = dsf.dims['year']
nmon

70

In [9]:
boxPhi = xr.Dataset()
#for ii in range(9):
for ii in range(13):
    print(ii)
    phi_box1 = np.ma.zeros([len(lme), nmon])
    phi_box1.mask = True
    phi_box2 = phi_box1.copy()
    for box,hh in zip(lme, range(len(lme))):
        latloc, lonloc = np.where(lme_mask == box)
        ar1 = np.ma.zeros([len(latloc),nmon]);ar1.mask=True
        ar2 = ar1.copy()
        for bb in range(len(latloc)):
            ar1[bb] = dsf[f'Phi_{ii}_0-200m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
            ar2[bb] = dsf[f'Phi_{ii}_200-600m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
        phi_box1[hh] = np.nanmean(ar1,axis=0)
        phi_box2[hh] = np.nanmean(ar2,axis=0)
    #boxPhi[f'Phi_{ii}_0-200m'] = xr.DataArray(data=phi_box1, coords=dict(lme=('lme', lme), time=('time', dsf.coords['time'].values)), dims=('lme','time'))
    #boxPhi[f'Phi_{ii}_200-600m'] = xr.DataArray(data=phi_box2, coords=dict(lme=('lme', lme), time=('time', dsf.coords['time'].values)), dims=('lme','time'))
    boxPhi[f'Phi_{ii}_0-200m'] = xr.DataArray(data=phi_box1, coords=dict(lme=('lme', lme), time=('year', dsf.coords['year'].values)), dims=('lme','year'))
    boxPhi[f'Phi_{ii}_200-600m'] = xr.DataArray(data=phi_box2, coords=dict(lme=('lme', lme), time=('year', dsf.coords['year'].values)), dims=('lme','year'))

0
1
2
3
4
5
6
7
8
9
10
11
12


In [10]:
boxPhi.load()

<xarray.Dataset>
Dimensions:          (lme: 11, year: 70)
Coordinates:
  * lme              (lme) int64 1 2 3 4 5 6 7 8 9 10 65
    time             (year) int64 0 1 2 3 4 5 6 7 8 ... 62 63 64 65 66 67 68 69
Dimensions without coordinates: year
Data variables: (12/26)
    Phi_0_0-200m     (lme, year) float64 0.6044 0.6001 0.6101 ... 0.5924 0.6166
    Phi_0_200-600m   (lme, year) float64 0.1084 0.105 0.104 ... 0.101 0.1033
    Phi_1_0-200m     (lme, year) float64 0.7104 0.7043 0.7135 ... 0.6805 0.7076
    Phi_1_200-600m   (lme, year) float64 0.1263 0.1223 0.1208 ... 0.1167 0.1193
    Phi_2_0-200m     (lme, year) float64 0.8363 0.8281 0.8358 ... 0.7822 0.8126
    Phi_2_200-600m   (lme, year) float64 0.1471 0.1424 0.1404 ... 0.1348 0.1378
    ...               ...
    Phi_10_0-200m    (lme, year) float64 3.264 3.195 3.125 ... 2.511 2.438 2.504
    Phi_10_200-600m  (lme, year) float64 0.5012 0.4838 0.4682 ... 0.4274 0.4361
    Phi_11_0-200m    (lme, year) float64 3.896 3.808 3.708 ... 2.899 2.818 2.888
    Phi_11_200-600m  (lme, year) float64 0.5846 0.5642 0.5446 ... 0.4938 0.5038
    Phi_12_0-200m    (lme, year) float64 4.656 4.544 4.407 ... 3.349 3.258 3.332
    Phi_12_200-600m  (lme, year) float64 0.682 0.658 0.6334 ... 0.5705 0.5819

In [11]:
dout = f'/glade/scratch/{USER}/DPLE-results'
#boxPhi.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_LME_1948-2017_monthly.nc', mode='w')
boxPhi.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_Eo_space_LME_1948-2017_monthly.nc', mode='w')